# Coursera Capstone Project
## By Akash Nauhwar


## Battle of Neighborhoods- Mumbai Location

In [25]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [26]:
# Shenton Way, District 01, Singapore
address = 'Dalal Street, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of India home are {}, {}.'.format(latitude, longitude))

C:\Users\vishalshivhare\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of India home are 18.929456, 72.8330142.


In [27]:
neighborhood_latitude=18.929456
neighborhood_longitude=72.8330142

In [28]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'N3ZEHLLYSNG5PZDCCVNXJDBYF45V0PJOBWQ22RGPOTBD55YL', 
    'TONUC031ELDBU0QNTEDNL1XQODIIDGBOCVYSRTXPST2H12HI', 
    '20180604', 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=N3ZEHLLYSNG5PZDCCVNXJDBYF45V0PJOBWQ22RGPOTBD55YL&client_secret=TONUC031ELDBU0QNTEDNL1XQODIIDGBOCVYSRTXPST2H12HI&v=20180604&ll=18.929456,72.8330142&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
#results

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

C:\Users\vishalshivhare\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Trishna,Seafood Restaurant,18.928619,72.832356
1,La Folie Patisserie,Dessert Shop,18.928504,72.832315
2,Filter,Gift Shop,18.928312,72.832643
3,Jehangir Art Gallery,Art Gallery,18.927606,72.831464
4,The Nutcracker,Breakfast Spot,18.928509,72.833683
5,The Irish House,Bar,18.927797,72.832295
6,Golden Dragon,Chinese Restaurant,18.928479,72.832493
7,Burma Burma,Asian Restaurant,18.929590,72.832043
8,H Parmesha Fast Food Corner,Fast Food Restaurant,18.930392,72.833549
9,Kala Ghoda Café,Café,18.928515,72.832354


## Map of Mumbai with venues near residence place - for reference 

In [24]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg